# Parámetros

In [11]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/AlwaysOn/data'
FILE_NAME = 'op.csv'
GROUP_BY_STORE = True
TOP_N_CATEGORIES = 5
GREATER_THAN_FR = 0.9
ACUMSUM = 0.9
MORE_THAN_ORDERS = 3


# Código

In [12]:
from pathlib import Path
from pandas import DataFrame, read_csv, cut

class AlwaysOn:
    def __init__(self, base_dir: str, file_name: str) -> None:
        # Guarda el directorio como atributo (tipo Path para combinar directorios, buscar archivos, etc)
        self.base_dir = Path(base_dir)
        # Guarda el nombre del archivo como atributo
        self.file_name = file_name
        # Crea el directorio completo del archivo, uniendo los dos atributos anteriores
        self.file_path = self.base_dir.joinpath(self.file_name)
        # Comprueba que el archivo exista, de no ser así imprime el error
        if not self.file_path.is_file():
            print(f'Debería existir un archivo llamado {self.file_name} en el directorio:\n{self.base_dir}\n\nAgrega el archivo e intenta de nuevo!')
            return None
        self.df = read_csv(self.file_path, low_memory=False)
        
    
    def top_n(self,  group_col: str, count_col: str, n: int) -> None:
        # Resetea el índice para ocupar dicha columna auxiliar en el siguiente paso
        self.top = self.df.reset_index()
        # Reestructura para contar las veces que se pide X producto a Y nivel (tienda, categoría de tienda, etc)
        self.top = self.top.pivot_table(index=[group_col,count_col], values='index', aggfunc='count').reset_index()
        # Calcula el ranking de mayor a menor a Y nivel y lo agrega como columna
        self.top[f'rank_{count_col}'] = self.top.groupby(group_col)['index'].rank(method="first", ascending=False).to_frame()
        # Filtra el self.top N
        self.top = self.top[self.top[f'rank_{count_col}']<=n].copy()
        # Ordena el resultado y elimina la columna auxiliar
        self.top = self.top.sort_values([group_col,f'rank_{count_col}']).drop('index', axis=1)


    def merge_and_group(self, **kwargs) -> None:
        # Unir info orders_product con el top categorías
        self.df = self.df.merge(self.top)
        # Reagrupar la info al nivel que se desea
        self.df = self.df.pivot_table(**kwargs).reset_index()
        

    def filter_fr(self, qty_found_col: str, qty_col: str, fr_limit: float, fr_bins: list) -> None:
        # Cálculo de Found Rate
        self.df['fr'] = self.df[qty_found_col] / (self.df[qty_col]+1e-10)
        # Máximo FR de 100%
        self.df['fr'] = self.df['fr'].map(lambda x: 1 if x > 1 else x)
        # Delimita los datos a sólo los que están sobre el límite de FR
        self.df = self.df[self.df['fr']>=fr_limit].copy()
        # Crea bines de FR, es decir: 47% -> (0, 0.7), 87% -> (0.8,0-9), etc
        self.df['fr_range'] = cut(self.df['fr'], bins=fr_bins)
        # Transforma el bin a texto: (0, 0.7) -> "00 - 70", (0.8,0-9) -> "80 - 90"
        self.df['fr_range'] = self.df['fr_range'].map(lambda x: str(int((x.left+.01)*100)).zfill(2)+' - '+str(int(x.right*100)).zfill(2))


    def filter_cumsum(self, acum_by: str, prod_cat_col: str, qty_col: str, cumsum_limit: float) -> None:
        # Ordenar la tabla de mayor a menor pero separado por Y nivel
        self.df.sort_values([acum_by, prod_cat_col, qty_col], ascending=[True, True, False], inplace=True)
        # Es necesario para ir acumulando la suma pero separado por Y nivel
        self.df['cumsum'] = self.df.groupby([acum_by, prod_cat_col])[qty_col].transform('sum')
        # Para después calcular el % de acumulado
        self.df['cumsum']= self.df[qty_col] / self.df['cumsum']
        # Y finalmente, acumular dicha suma %
        self.df['cumsum'] = self.df.groupby([acum_by,prod_cat_col])['cumsum'].cumsum()
        # Delimita los datos a sólo los que están debajo del límite acumulado
        self.df = self.df[self.df['cumsum']<=cumsum_limit].copy()

    def filter_p_orders(self, qty_col: str, qty_limit: float) -> None:
        # Delimita los datos a sólo los que se pidieron P veces o más
        self.df = self.df[self.df[qty_col]>=qty_limit].copy()

# Importar

In [13]:
ao = AlwaysOn(BASE_DIR, FILE_NAME)
print(ao.df.shape)
ao.df.head()

(12045001, 10)


,Unnamed: 0,Store Category Name (Spanish),Store ID,Store Name,Product ID,Name,Category (Spanish),Order ID,Sum Qty Products Ordered,Sum Qty Products Found
0,1,Panadería y pastelería,14700.0,Quequeríaa Receitas Pup´s,5875486.0,Caja trocitos de brownie,Queques,58758439.0,1.0,1.0
1,2,Panadería y pastelería,14700.0,Quequeríaa Receitas Pup´s,5876051.0,Trozo queque del día,Queques,58758439.0,1.0,1.0
2,3,Panadería y pastelería,14700.0,Quequeríaa Receitas Pup´s,5876057.0,Cholates crackers,Chocolate,58758439.0,1.0,1.0
3,4,Gourmet,14669.0,Emporio Gorroño,5864231.0,Queso de oveja maduro,Quesos especiales,58523457.0,1.0,1.0
4,5,Gourmet,14669.0,Emporio Gorroño,5864109.0,Edamame coreano,Frutas y verduras congeladas,58571610.0,1.0,1.0


# Top N categorías

In [14]:
ao.top_n(group_col='Store ID' if GROUP_BY_STORE else 'Store Category Name (Spanish)', count_col='Category (Spanish)', n=TOP_N_CATEGORIES)
print(ao.top.shape)
ao.top.head()

(14076, 3)


,Store ID,Category (Spanish),rank_Category (Spanish)
14,1638.0,Ojos,1.0
16,1638.0,Rostro,2.0
0,1638.0,Accesorios belleza,3.0
17,1638.0,Uñas,4.0
11,1638.0,Limpieza y cuidado cabello,5.0


# Unir y agrupar

In [15]:
TO_GROUP = ['Store Category Name (Spanish)', 'Store ID', 'Store Name', 'Product ID', 'Name', 'Category (Spanish)', 'rank_Category (Spanish)'] if GROUP_BY_STORE else ['Store Category Name (Spanish)', 'Product ID', 'Name', 'Category (Spanish)', 'rank_Category (Spanish)']

ao.merge_and_group(
    index=TO_GROUP, 
    values=['Sum Qty Products Ordered', 'Sum Qty Products Found'], 
    aggfunc=sum
    )

print(ao.df.shape)
ao.df.head()

(226446, 9)


,Store Category Name (Spanish),Store ID,Store Name,Product ID,Name,Category (Spanish),rank_Category (Spanish),Sum Qty Products Found,Sum Qty Products Ordered
0,Belleza y cuidado personal,1638.0,DBS Beauty Store,150773.0,Alicate Cutículas de Acero Inoxidable,Accesorios belleza,3.0,1.0,1.0
1,Belleza y cuidado personal,1638.0,DBS Beauty Store,270693.0,ACEITE REPARADOR MACADAMIA,Limpieza y cuidado cabello,5.0,1.0,1.0
2,Belleza y cuidado personal,1638.0,DBS Beauty Store,270700.0,ACONDICIONADOR DE ARGAN,Limpieza y cuidado cabello,5.0,1.0,1.0
3,Belleza y cuidado personal,1638.0,DBS Beauty Store,375635.0,Acondicionador argan reparación,Limpieza y cuidado cabello,5.0,1.0,1.0
4,Belleza y cuidado personal,1638.0,DBS Beauty Store,375637.0,Acondicionador coconut revitalizador e hidratante,Limpieza y cuidado cabello,5.0,2.0,2.0


# Límite de FR

In [16]:
ao.filter_fr(qty_found_col='Sum Qty Products Found', qty_col='Sum Qty Products Ordered', fr_limit=GREATER_THAN_FR, fr_bins=[-.01,0.7,0.8,0.9,0.95,1])
print(ao.df.shape)
ao.df.head()

(166747, 11)


,Store Category Name (Spanish),Store ID,Store Name,Product ID,Name,Category (Spanish),rank_Category (Spanish),Sum Qty Products Found,Sum Qty Products Ordered,fr,fr_range
0,Belleza y cuidado personal,1638.0,DBS Beauty Store,150773.0,Alicate Cutículas de Acero Inoxidable,Accesorios belleza,3.0,1.0,1.0,1.0,96 - 100
1,Belleza y cuidado personal,1638.0,DBS Beauty Store,270693.0,ACEITE REPARADOR MACADAMIA,Limpieza y cuidado cabello,5.0,1.0,1.0,1.0,96 - 100
2,Belleza y cuidado personal,1638.0,DBS Beauty Store,270700.0,ACONDICIONADOR DE ARGAN,Limpieza y cuidado cabello,5.0,1.0,1.0,1.0,96 - 100
3,Belleza y cuidado personal,1638.0,DBS Beauty Store,375635.0,Acondicionador argan reparación,Limpieza y cuidado cabello,5.0,1.0,1.0,1.0,96 - 100
4,Belleza y cuidado personal,1638.0,DBS Beauty Store,375637.0,Acondicionador coconut revitalizador e hidratante,Limpieza y cuidado cabello,5.0,2.0,2.0,1.0,96 - 100


# Top M% de productos

In [17]:
ao.filter_cumsum(acum_by='Store ID' if GROUP_BY_STORE else 'Store Category Name (Spanish)', prod_cat_col='Category (Spanish)', qty_col='Sum Qty Products Ordered', cumsum_limit=ACUMSUM)
print(ao.df.shape)
ao.df.head()

(107115, 12)


,Store Category Name (Spanish),Store ID,Store Name,Product ID,Name,Category (Spanish),rank_Category (Spanish),Sum Qty Products Found,Sum Qty Products Ordered,fr,fr_range,cumsum
141,Belleza y cuidado personal,1638.0,DBS Beauty Store,1606521.0,Lima negra curva 100-180,Accesorios belleza,3.0,17.0,17.0,1.0,96 - 100,0.097701
202,Belleza y cuidado personal,1638.0,DBS Beauty Store,1903581.0,Repujador de cuticulas,Accesorios belleza,3.0,8.0,8.0,1.0,96 - 100,0.143678
58,Belleza y cuidado personal,1638.0,DBS Beauty Store,1380307.0,Encrespador,Accesorios belleza,3.0,7.0,7.0,1.0,96 - 100,0.183908
138,Belleza y cuidado personal,1638.0,DBS Beauty Store,1605983.0,Corta cuticulas,Accesorios belleza,3.0,7.0,7.0,1.0,96 - 100,0.224138
145,Belleza y cuidado personal,1638.0,DBS Beauty Store,1606528.0,Lima block,Accesorios belleza,3.0,7.0,7.0,1.0,96 - 100,0.264368


# Filtrar P órdenes

In [18]:
ao.filter_p_orders(qty_col='Sum Qty Products Ordered', qty_limit=MORE_THAN_ORDERS)
print(ao.df.shape)
ao.df.head()

(44955, 12)


,Store Category Name (Spanish),Store ID,Store Name,Product ID,Name,Category (Spanish),rank_Category (Spanish),Sum Qty Products Found,Sum Qty Products Ordered,fr,fr_range,cumsum
141,Belleza y cuidado personal,1638.0,DBS Beauty Store,1606521.0,Lima negra curva 100-180,Accesorios belleza,3.0,17.0,17.0,1.0,96 - 100,0.097701
202,Belleza y cuidado personal,1638.0,DBS Beauty Store,1903581.0,Repujador de cuticulas,Accesorios belleza,3.0,8.0,8.0,1.0,96 - 100,0.143678
58,Belleza y cuidado personal,1638.0,DBS Beauty Store,1380307.0,Encrespador,Accesorios belleza,3.0,7.0,7.0,1.0,96 - 100,0.183908
138,Belleza y cuidado personal,1638.0,DBS Beauty Store,1605983.0,Corta cuticulas,Accesorios belleza,3.0,7.0,7.0,1.0,96 - 100,0.224138
145,Belleza y cuidado personal,1638.0,DBS Beauty Store,1606528.0,Lima block,Accesorios belleza,3.0,7.0,7.0,1.0,96 - 100,0.264368


In [19]:
# Cantidad de productos únicos
len(set(ao.df['Product ID']))

36896

# Exportar

In [20]:
ao.df.to_csv(ao.base_dir.joinpath(f'always_on_store{"" if GROUP_BY_STORE else "_category"}.csv'), index=False, sep='\t', encoding='utf-16')